# Scripted Chatbots
![Rule based bot](https://theappsolutions.com/images/articles/source/chatbot-for-travel-business/rule-based-catbots.png)

Chatbots typically follow ** Query Response Model**. Pattern Matching  helps the bots in a great way to handle the incoming requests from the user and providing an appropriate response . Interactions can also be structured as shown in the Figure below. Here, the user is provided with list of services offered by the company. Henceforth, Structuring the interaction will help in yielding better responses

## Task 1: Lets Implement a simple greeting bot

It checks whether a incoming request is a greeting by string matching technique.
List of greeting words are stored. Incoming request is searched for the presence of a greeting word. If present, the bot provides a greeting response.
<br/>
![Greeting Bot](https://images.twinkl.co.uk/tw1n/image/private/t_630/image_repo/7b/2f/T-S-1961-Social-Greetings-Prompt-Cards.jpg)




In [0]:
import random

GREETING_KEYWORDS = ["hello", "hi", "morning", "whats up", "greetings"]
GREETING_RESPONSE = ["let ur day be as bright as your sun", "hello", "have a pleasant day"]
def check_for_greeting(sentence):
  words = sentence.split(' ')
  for word in words:
    if word.lower() in GREETING_KEYWORDS:
      return random.choice(GREETING_RESPONSE)
  return "I don't understand"

In [0]:
# Write a statement and check the response from the greeting bot
input_query="hi have a good day"
print(check_for_greeting(input_query))

have a pleasant day


## Task 2: Eliza

Integral component of Eliza is pattern matching.
<br/>
Regular expression are used for extracted key components in the incoming request which then used in providing an appropriate response![alt text](https://miro.medium.com/max/2702/0*sawrTdROOtCkFSXA.)

In [0]:
import re
pattern ='What if(.*)' # In regular expression . indicates any character * indicates one (or) more occurence of the same character
message = 'What if i ask you for help'

# search method in regular expression will help in identifying whether the pattern occurs in the message
# it returns a match object which is None when the pattern is not present in the message

match = re.match(pattern, message)
print(match)

<_sre.SRE_Match object; span=(0, 26), match='What if i ask you for help'>


In [0]:
# Function enables the message to be verified against series of patterns
# returns the first pattern to which message confirms
# If the pattern is not found, it returns None
def verify (message):
  patterns = ['heelo wo.*', 'unicode .*']
  for i in patterns:
    match = re.match(i, message)
    if match:
      return i
  return None

In [0]:
print(verify('heelo world'))
print(verify('h'))

heelo wo.*
None


*match method* checks and returns whether the message conforms to a given pattern. More than matching patterns, it is also important to extract key phrases in the message.<br/>

**Eliza uses the key phrases to create the meaningful responses**
<br/>


*   To extract key phrases, Groups are used in regular expression.
*   Adding paranthesis in the pattern indicates a group.





In [0]:
# The following pattern can be used to identify the series of characters that occur before if and after if in a message
pattern = '(.*) if (.*)'
message = 'God if win Demon if fail'

match = re.search(pattern, message)
# group (0) will provide the entire message
# group (1) will extract the characters before if
# group (2) will extract the characters after if

print(match.group(0))
print(match.group(1))
print(match.group(2))



God if win Demon if fail
God if win Demon
fail


Next important concept in Eliza is **Grammatical Transformation** of the key phrases extracted. To make the responses grammatically coherent, first and second person to be transformed

In English, this can easily be achieved using conjugative verbs

Following method replace the pronouns upon invocation with the extracted key phrases

me -> you
<br/>
my -> your
<br/>
your -> my
<br/>
you -> me
<br/>



In [0]:
def replace_pronouns(message): 
  message = message.lower()
  if 'me' in message: 
    # sub method replace the occurence of str1 with str2 in the given message
    message = re.sub('me','you',message) 
    return message 
  if 'my' in message:
    message = re.sub('my','your',message) 
    return message 
  if 'your' in message: 
    message = re.sub('your','my',message) 
    return message 
  if 'you' in message: 
    message = re.sub('you','me',message) 
    return message 
  return message 

In [0]:
print(replace_pronouns("my last birthday")) 
print(replace_pronouns("when you went to Florida")) 
print(replace_pronouns("I had my own castle")) 

your last birthday
when me went to florida
i had your own castle


### Building a Complete Model

---




1.   Match Rules
2.   Replace Pronouns
3.   Compose Reply



In [0]:
# Creating a dict with many patterns
rules = {'if(.*)' : ['Do you really think its likely that {0}','What do you think about {0}','Really- if {0}'],
         'do you think (.*)' :['If {0} ?Absolutely.', 'No chance'],
         'I want (.*)': ['Why do you want {0}','Whats stopping you from getting {0}','What would it mean if you got {0}'],
         'do you remember (.*)': ['Did you think I would forget {0}', 'Why havent you been able to forget {0}','What about {0}','Yes.. And ?']
         }

def match_rule(rules, message): 
    response, phrase = "default", None 
    # Iterate over the rules dictionary 
    for pattern,responses in rules.items(): 
        # Create a match object 
        match = re.search(pattern, message) 
        if match is not None: 
            response = random.choice(responses) 
            # Choose a random response 
            if '{0}' in response: 
                phrase = match.group(1) 
    # Return the response and phrase 
    return response, phrase 

In [0]:
print(match_rule(rules, "do you remember your last birthday")) 
print(match_rule(rules,"how are you?"))

('Did you think I would forget {0}', 'your last birthday')
('default', None)


In [0]:
## Composing the reply
## match_rule will provide the response pattern and extracted key phrase
## then replace pronoun is called for grammatical transformation
## if the returning response pattern from match rule is default then a standard response is offered
## else, the output of replace pronoun is used to modify the response pattern suitably

def respond(message):
  response, phrase = match_rule(rules, message)
  standard_responses = ["Tell me more", "I donot Understand", "How can i help you?"]
  if response == 'default':
    return random.choice(standard_responses)
  else:
    modified_phrase = replace_pronouns(phrase)
    response = response.format(modified_phrase)
  return response

In [0]:
print(respond("do you remember your last birthday"))

Why havent you been able to forget my last birthday


In [0]:
print(respond("I want a robot friend"))

Why do you want a robot friend


In [0]:
print(respond("why cars have four tyres"))

I donot Understand


# Intent Recognizers

---


 From the natural language statements, infer the intent (or) the command of the user to the bot

 To understand the working of intent recognizers, let us consider the airline travel information system

The system provides services to book tickets, obtain flight related information, understand the air fare etc

In [0]:
# spacy is an advanced natural language processing in python
# it has pretrained statistical models and word vectors for language modeling
import spacy

In [0]:
# The dataset has a pickle file Which contains various user commands and intents
# They are available as vectors of numbers
# To decipher the numbers, dict which maps the numbers with tokens are provided
import pickle
def load_ds(fname='atis.train.pkl'):
  with open(fname,'rb') as stream:
    ds,dicts = pickle.load(stream)
  return ds,dicts

In [0]:
train_ds, dicts = load_ds('/content/atis.train.pkl')
print(type(train_ds))
print(type(dicts))

<class 'collections.defaultdict'>
<class 'dict'>


In [0]:
# tokens - words
# slot - entities
# intent
t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids','intent_ids'])
print(in2i)

{'abbreviation': 0, 'aircraft': 1, 'aircraft+flight+flight_no': 2, 'airfare': 3, 'airfare+flight': 4, 'airfare+flight_time': 5, 'airline': 6, 'airline+flight_no': 7, 'airport': 8, 'capacity': 9, 'cheapest': 10, 'city': 11, 'day_name': 12, 'distance': 13, 'flight': 14, 'flight+airfare': 15, 'flight+airline': 16, 'flight_no': 17, 'flight_no+airline': 18, 'flight_time': 19, 'ground_fare': 20, 'ground_service': 21, 'ground_service+ground_fare': 22, 'meal': 23, 'quantity': 24, 'restriction': 25}


In [0]:
i2t, i2s, i2in = map(lambda d: {d[k]:k for k in d.keys()}, [t2i,s2i,in2i])
print(i2in)

{0: 'abbreviation', 1: 'aircraft', 2: 'aircraft+flight+flight_no', 3: 'airfare', 4: 'airfare+flight', 5: 'airfare+flight_time', 6: 'airline', 7: 'airline+flight_no', 8: 'airport', 9: 'capacity', 10: 'cheapest', 11: 'city', 12: 'day_name', 13: 'distance', 14: 'flight', 15: 'flight+airfare', 16: 'flight+airline', 17: 'flight_no', 18: 'flight_no+airline', 19: 'flight_time', 20: 'ground_fare', 21: 'ground_service', 22: 'ground_service+ground_fare', 23: 'meal', 24: 'quantity', 25: 'restriction'}


In [0]:
query, slots, intent =  map(train_ds.get, ['query', 'slot_labels', 'intent_labels'])

In [0]:
# Creating a Training set
# Samples and their corresponding labels are stored in separate arrays
sample = []
labels = []
for i in range(4978):
    sentence = (' '.join(map(i2t.get, query[i])))
    sample.append(sentence)
    labels.append(i2in[intent[i][0]])

In [0]:
print(len(sample))
print(len(labels))

4978
4978


In [0]:
# User Command and their intent
print(sample[419])
print(labels[419])

BOS what airplane types fly from pittsburgh to baltimore EOS
aircraft


### Task 3: Creating a Supervised Machine Learning Model to recognize the intent of the bot

In [0]:
# The statements cannot be understand directly from the machine learning model
# words have to be converted into vectors
# spacy has word vectors for every possible word in thesaurus constructed using glove algorithm
# Each word is coded as three hundred dimensional vector
!python -m spacy download en_core_web_lg


     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=741640572510ec67b250edd8de37e935d75c2c846a9d4c079a51c1bbb1c92d33
  Stored in directory: /tmp/pip-ephem-wheel-cache-zphqksti/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load() # load the Glove vectors for words


In [0]:
print(nlp.vocab.vectors_length) # It provides a three dimensional vector

300


#### Understanding the Glove Vectors

In [0]:
vec1 = nlp("foot")
vec2 = nlp("football")
vec3 = nlp("cricket")

print(vec1.similarity(vec2))
print(vec2.similarity(vec3))

0.2559363414015776
0.5254916173923929


#### Nearest Neighbor Classifier for Intent Classification

In [0]:
import numpy as np
# Create a training set
# from the list of sample utterances
X_train_shape = (len(sample),nlp.vocab.vectors_length)
X_train = np.zeros(X_train_shape)
for sentence in sample:
    X_train[i,:] = nlp(sentence).vector

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
# Sample test utterance for which intent has to be identified
test_message = """i would like to find a flight from charlotteto las vegas that makes a stop in st. louis"""

In [0]:
# Obtain the Glove Vector for the test utterance
test_x = nlp(test_message).vector
# Reshape the vector to size 1*300
test_x =test_x.reshape(1,300)

In [0]:
#Compute Cosine similarity with respect to all the training samples
scores = []
for i in range(len(sample)):
    y= X_train[i,:].reshape(1,300)
    scores.append(cosine_similarity(y, test_x)[0])

In [0]:
# Identify the sample with maximum similarity
scores = np.array(scores)
print(np.argmax(scores))

# Identify the label associated with the sample
print(labels[np.argmax(scores)])

0
flight


#### Eager Classifier for Intent Classification

In [0]:
# Building one vs rest classifier for deriving non-linear model for identifying the best hyperplane 
# that can discriminate the vectors of various classes in an optimal way
from sklearn.svm import SVC
clf=SVC()
clf.fit(X_train, labels)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(test_x)
print(y_pred)

['flight']


### Task 4: Identification of entities and extraction of roles

In [0]:
doc = nlp('show me flights from India to Shanghai')
# Named Entity Recognition uses contextual clues to identify 
for ent in doc.ents:
  print(ent.text, ent.label_)
# GPE stands for Geo political entity which includes countries, cities and states

India GPE
Shanghai GPE


In [0]:
# Visualizing Dependency Parsing

from spacy import displacy
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})